In [24]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as pyplot
import numpy as np

#**Assignment 2: Convolutional Neural Network**

##**Data**

Data: CIFAR-10

CIFAR-10  is an established computer-vision dataset used for object recognition. It is a subset of the 80 million tiny images dataset and consists of 60,000 32x32 **color images** containing one of 10 object classes, with 6000 images per class.

Labels are as follows:

airplane (0), automobile (1), bird (2), cat (3), deer (4), dog (5), frog (6), horse (7), ship (8), truck (9)

Source: https://www.kaggle.com/c/cifar-10

### Q1. This dataset has been included in **keras.datasets.cifar10**. Please load the dataset and print the shape of training and testing sets.

In [25]:
# load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
# print the shape of training and testing sets
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (50000, 32, 32, 3)
Testing set shape: (10000, 32, 32, 3)


### Q2. Convert the target labels (y) into the one-hot format and show the value (y) for the first instance of the training dataset.

In [26]:
# convert target labels to one-hot format
y_train_one_hot = keras.utils.to_categorical(y_train, num_classes=10)
y_test_one_hot = keras.utils.to_categorical(y_test, num_classes=10)
# show the value for the first instance of the training dataset
print("First instance of training labels (one-hot):", y_train_one_hot[0])

First instance of training labels (one-hot): [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


### Q3. Create a validation dataset using the first 5,000 instances in the training dataset. Also, divide all input features (X values) in the train/test/validation sets by 255.0. Please show the y value for the first instance of the validation dataset.

In [27]:
# create validation dataset
X_val = X_train[:5000] / 255.0
y_val = y_train_one_hot[:5000]
X_train = X_train[5000:] / 255.0
y_train_one_hot = y_train_one_hot[5000:]
X_test = X_test / 255.0
# show the y value for the first instance of the validation dataset
print("First instance of validation labels (one-hot):", y_val[0])

First instance of validation labels (one-hot): [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


## **Model**

### Q4. Create a convolutional neural network with 32 kernels of size 3 by 3 in the first layer and 64 kernels of size 3 by 3 in the second layer. We need a max pooling layer wth the size of 2 after each convolution layer. After flattening the feature maps add a fully-connected layer with 128 nodes for the final prediction. Please print the model summary.

### Please note that the input images are color images with the shape of **32 * 32 * 3**. Here 3 shows RGB.

In [28]:
# create the model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3),padding='same'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu',padding='same'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
# print the model summary
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_14 (Conv2D)              │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 545,098 (2.08 MB)

 Trainable params: 545,098 (2.08 MB)

 Non-trainable params: 0 (0.00 B)

### Q5. Create an Adam optimizer with a learning rate of 0.001, compile the model, and fit it on the training and validation datasets. Use the following hyperparameters: **batch_size=512, epochs=5**.

### *Hint*: Adam optimizer can be imported by **keras.optimizers.Adam(learning_rate=0.001)**

In [29]:
# create Adam optimizer
optimizer = keras.optimizers.Adam(learning_rate=0.001)
# compile the model
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# fit the model
model.fit(X_train, y_train_one_hot,
          batch_size=512,
          epochs=5,
          validation_data=(X_val, y_val))

Epoch 1/5
88/88 ━━━━━━━━━━━━━━━━━━━━ 10s 106ms/step - accuracy: 0.3795 - loss: 1.7355 - val_accuracy: 0.4976 - val_loss: 1.4435
Epoch 2/5
88/88 ━━━━━━━━━━━━━━━━━━━━ 10s 108ms/step - accuracy: 0.5117 - loss: 1.3787 - val_accuracy: 0.5548 - val_loss: 1.2678
Epoch 3/5
88/88 ━━━━━━━━━━━━━━━━━━━━ 10s 109ms/step - accuracy: 0.5743 - loss: 1.2233 - val_accuracy: 0.5828 - val_loss: 1.1764
Epoch 4/5
88/88 ━━━━━━━━━━━━━━━━━━━━ 10s 114ms/step - accuracy: 0.6073 - loss: 1.1315 - val_accuracy: 0.6138 - val_loss: 1.1217
Epoch 5/5
88/88 ━━━━━━━━━━━━━━━━━━━━ 10s 115ms/step - accuracy: 0.6292 - loss: 1.0637 - val_accuracy: 0.6188 - val_loss: 1.0733


## **Performance**

### Q6. Evaluate your model.

In [30]:
# evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot)
print("Test accuracy:", test_accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6201 - loss: 1.0761
Test accuracy: 0.6201000213623047


## **Post-analysis**

### Q7. Print the name of layers.

#### Note: You may choose any name you like for your layers.

In [31]:
# print the name of layers
for layer in model.layers:
    print(layer.name)

conv2d_14
max_pooling2d_14
conv2d_15
max_pooling2d_15
flatten_7
dense_14
dense_15


### Q8. Print the output of layers.

In [37]:
# Ensure the model is built by calling it with a dummy input
dummy_input = tf.zeros((1, 32, 32, 3))  # Shape matches the input shape of the model
model(dummy_input)

# Create a new model that outputs the intermediate layers
layer_outputs = [layer.output for layer in model.layers]
activation_model = keras.Model(inputs=model.input, outputs=layer_outputs)

# Get the outputs for the first test instance
activations = activation_model(X_test[4:5])

# Print the output of each layer
for i, activation in enumerate(activations):
    print(f"Output of layer {i} ({model.layers[i].name}):")
    print(activation.numpy())

AttributeError: The layer sequential_7 has never been called and thus has no defined input.

### Q9. Print the output shape of feature maps in the convolution layers as well as the pooling layers for the fifth instance in your test dataset.

The frog image below should be your fifth testing instance (index=4).

### Q10. Draw 16 feature maps for each convolution and pooling layer.